In [1]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 72.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source 

In [2]:
import transformers 
import torch

import pandas as pd

In [3]:
# model_id = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B" 
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

pipeline = transformers.pipeline( 
"text-generation",  
model=model_id,  
max_new_tokens=2048,  
model_kwargs={"dtype": torch.bfloat16},  
device_map="auto" 
)

2025-11-08 06:55:43.579427: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762584943.984189      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762584944.106526      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [4]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.
### Instruction: 
You are an expert of film comment analysis. Analyze the given text from an online review and determine the sentiment polarity. Return a single number of either 0 and 1, with 0 being negative and 1 being the positive sentiment. Return only 0 (negative) or 1 (positive).
### Input: {}
### Response:
<think>
"""

In [5]:
test = pd.read_csv("/kaggle/input/corpus-imdb/testData.tsv", header=0, delimiter="\t", quoting=3)


In [6]:
from tqdm.auto import tqdm
import re
from datasets import Dataset

# 创建带进度显示的pipeline包装器
def pipeline_with_progress(pipeline, prompts, batch_size=4, **kwargs):
    total_batches = (len(prompts) + batch_size - 1) // batch_size
    
    # 分批处理并显示进度
    all_responses = []
    for batch_idx in tqdm(range(total_batches), desc="inference"):
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, len(prompts))
        batch_prompts = prompts[start_idx:end_idx]
        
        batch_responses = pipeline(
            batch_prompts,
            batch_size=len(batch_prompts),
            **kwargs
        )
        all_responses.extend(batch_responses)
    
    return all_responses

def format_prompt(text):
    text["prompt"] = prompt_style.format(text["review"])
    return text

test_dataset = Dataset.from_pandas(test)
test_dataset = test_dataset.map(format_prompt)
prompt_list = list(test_dataset["prompt"])


# 使用tqdm显示进度
test_pred = []
total_samples = len(prompt_list)

responses = pipeline_with_progress(
    pipeline,
    prompt_list,
    batch_size=1,
    do_sample=False,
    pad_token_id=pipeline.tokenizer.eos_token_id,
    max_new_tokens=3,
)

for i, response in enumerate(tqdm(responses, desc="analysis", total=total_samples)):
    output = response[0]["generated_text"]
    
    # 提取数字
    digits = re.findall(r'[01]', output)
    pred = int(digits[-1]) if digits else 0
    test_pred.append(pred)
    


result_output = pd.DataFrame(data={"id": test["id"], "sentiment": test_pred})
result_output.to_csv("/kaggle/working/instruction_learning.csv", index=False, quoting=3)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

inference:   0%|          | 0/25000 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


analysis:   0%|          | 0/25000 [00:00<?, ?it/s]